<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Workflow" data-toc-modified-id="Workflow-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Workflow</a></span></li><li><span><a href="#Inhabitants-data" data-toc-modified-id="Inhabitants-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Inhabitants data</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Confirmed-cases-(Sciensano)" data-toc-modified-id="Confirmed-cases-(Sciensano)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Confirmed cases (Sciensano)</a></span></li><li><span><a href="#Time-series-comparison" data-toc-modified-id="Time-series-comparison-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Time series comparison</a></span><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Granger-causality" data-toc-modified-id="Granger-causality-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Granger causality</a></span></li><li><span><a href="#Dynamic-time-warping" data-toc-modified-id="Dynamic-time-warping-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Dynamic time warping</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
from scipy import stats

# Workflow

- compare timeseries of confirmed cases, deaths and hospitalisations for the 43 arrondissements
    - Granger Causality Test: if prediction of one time series is improved by incorporating the information of a second time series, the latter has causal influence on the former. If this is not the case: significantly different behaviour.
    - Dynamic Time Warping Distance: measure for alignment between two given time series
    - autocorrelation tests?
    - model fit ARMA... ?
    - ...
- check how much transmission there is between arrondissements (commuter data)
    - data will be provided by VITO

In the current version of the notebook, public data of sciensano are used (https://epistat.sciensano.be/covid/covid19_historicaldata.html). Note that only confirmed cases are available per municipality (aggregated below per arrondissement). Hospitalisation are only available per province, mortality data are only available per region (i.e. Flanders/Wallonia/Brussels). Also, the data are only available from 31 March onwards. Hence, for more detailed and more extended data we have to wait for data access.

# Inhabitants data

In [2]:
# read nb of inhabitants per municipality
url_inhabitants = 'https://statbel.fgov.be/sites/default/files/files/documents/bevolking/5.1%20Structuur%20van%20de%20bevolking/Bevolking_per_gemeente.xlsx'
inhabitants = pd.read_excel(url_inhabitants, skipfooter=4, skiprows=1).dropna(how='all', axis=0)
inhabitants['NIS code'] = inhabitants['NIS code'].astype(int)

In [3]:
inhabitants[inhabitants.Woonplaats.str.startswith('Arrondissement')].head()

,NIS code,Woonplaats,Mannen,Vrouwen,Totaal
7,11000,Arrondissement Antwerpen,524320.0,533416.0,1057736.0
38,12000,Arrondissement Mechelen,170620.0,176505.0,347125.0
51,13000,Arrondissement Turnhout,232833.0,232036.0,464869.0
81,21000,Arrondissement Brussel-Hoofdstad,597706.0,620549.0,1218255.0
101,23000,Arrondissement Halle-Vilvoorde,313872.0,329894.0,643766.0


# Functions

In [4]:
def update_local_data(datelist):
    """
    Function to update infection data from Sciensano website that have not yet been saved locally
    
    Parameters:
    -----------
    datelist : list
        list of data in format 'YYYYMMDD', e.g. '20200331'
    """
    
    local_list = []
    for filename in glob.glob('../data/cumulative_cases_per_community/'+'COVID19BE_CASES_MUNI_CUM_*.csv'):
        date = filename.split('/')[-1].split('_')[-1].split('.csv')[0]
        local_list = local_list+[date]
    missing_dates = set(datelist) - set(local_list)
    for date in missing_dates:
        url = 'https://epistat.sciensano.be/Data/'+ date +'/COVID19BE_CASES_MUNI_CUM_'+ date +'.csv'
        cases_day_x = pd.read_csv(url, encoding = "ISO-8859-1", skipfooter=1, engine='python').dropna(axis=0, how='all').copy()
        cases_day_x.to_csv('../data/cumulative_cases_per_community/'+'COVID19BE_CASES_MUNI_CUM_'+ date +'.csv')
    return

In [5]:
def prepare_arr_cases(date, read_local = True):
    """
    Read number of infected cases and merge per arrondissement
    
    Parameters
    ---------
    date : string
        date for which to read infected cases, in format 'YYYYMMDD', e.g. '20200331'
    read_local : boolean
        True if reading locally saved data, False if reading directly from Sciensano server
    
    """
    if read_local == True:
        cases_day_x = pd.read_csv('../data/cumulative_cases_per_community/'+'COVID19BE_CASES_MUNI_CUM_'+ date +'.csv')
    else:
        url = 'https://epistat.sciensano.be/Data/'+ date +'/COVID19BE_CASES_MUNI_CUM_'+ date +'.csv'
        cases_day_x = pd.read_csv(url, encoding = "ISO-8859-1", skipfooter=1, engine='python').dropna(axis=0, how='all').copy()
        # engine = 'python' to avoid skipfooter warning
    cases_day_x.NIS5 = cases_day_x.NIS5.astype(int)
    cases_day_x = cases_day_x[['TX_ADM_DSTR_DESCR_NL','CASES']].copy()
    # Assume <5 cases to be 3 cases
    cases_day_x.loc[cases_day_x.CASES.str.find('<')!=-1, 'CASES'] = '3'
    cases_day_x.CASES = cases_day_x.CASES.astype(int)
    cases_arr = cases_day_x.groupby(by='TX_ADM_DSTR_DESCR_NL')['CASES'].sum().reset_index()
    cases_arr = cases_arr.set_index('TX_ADM_DSTR_DESCR_NL')
    return cases_arr

# Confirmed cases (Sciensano)

In [6]:
datelist = pd.date_range(start = pd.to_datetime('2020-03-31'), end = datetime.today() - timedelta(days=1)).strftime("%Y%m%d")

In [ ]:
## to save Sciensano data locally (anticipating any change in open data policy at Sciensano)
update_local_data(datelist)

In [7]:
## Collect infected cases in a dictionary, for all dates in datelist
ts_arr_cases_dict = {}
for date in datelist:
    ts_arr_cases_dict[date] = prepare_arr_cases(date)
    # fix typo's in the data
    ts_arr_cases_dict[date] = ts_arr_cases_dict[date].rename({'Arrondissement NeufchÃ¢teau':'Arrondissement Neufchâteau',
                                      'Arrondissement La LouviÃ¨re':'Arrondissement La Louvière'})


In [8]:
## convert dict to a dataframe with the timeseries of each arrondissement in columns
ts_arr_cum_cases = pd.concat(ts_arr_cases_dict, axis=1)
ts_arr_cum_cases.columns = ts_arr_cum_cases.columns.droplevel(level=1)
ts_arr_cum_cases = ts_arr_cum_cases.T
ts_arr_cum_cases.columns = [x.split('Arrondissement ')[-1] for x in ts_arr_cum_cases.columns]

In [9]:
## non-cumulative cases
ts_arr_cases = ts_arr_cum_cases.diff().dropna()

In [10]:
## nb of cases relative to inhabitants of that arrondissement
ts_arr_rel_cases = ts_arr_cases.apply(lambda col: ts_arr_cases[col.name]/inhabitants.loc[inhabitants.Woonplaats=='Arrondissement '+col.name, 'Totaal'].item(), axis=0)

# Time series comparison

Normality and stationarity are ignored here, since we work with time series (mostly not normal) and the trend (hence non-stationarity) is needed to compare the time series mutually.

## Functions

In [11]:
def granger_test(df, col1, col2, lag=1):
    """
    df : df with timeseries in columns
    col1, col2 : test whether col2 granger causes col1
    lag : timelag for which to test granger causality
    """
    data = df[[col1,col2]]
    # for now just 1 lag tested
    gc_res = grangercausalitytests(data, maxlag=[lag], verbose=False)
    stat = gc_res[lag][0]['ssr_ftest'][0]
    p = gc_res[lag][0]['ssr_ftest'][1]
    return stat,p

In [12]:
def test_df(test_function, timeseries_df, test_list, lag=1):
    """
    test_function : granger_test, ...
    timeseries_df : df with timeseries in columns
    test_list : list with column names to compare
    lag : only needed for granger_test, unused argument for other tests
    """
    stat_df = pd.DataFrame(index=test_list, columns=test_list)
    p_df = pd.DataFrame(index=test_list, columns=test_list)
    for col1 in test_list:
        for col2 in test_list:
            test = test_function(timeseries_df, col1, col2, lag=lag)
            stat_df[col1][col2] = test[0] # df[col][row] calls the element
            p_df[col1][col2] = test[1]
            
    return stat_df, p_df

## Granger causality

In [13]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.grangercausalitytests.html

https://www.statisticshowto.com/granger-causality/

The Null hypothesis for grangercausalitytests is that the time series in the second column, x2, does NOT Granger cause the time series in the first column, x1. Grange causality means that past values of x2 have a statistically significant effect on the current value of x1, taking past values of x1 into account as regressors. We reject the null hypothesis that x2 does not Granger cause x1 if the p-values are below a desired size of the test.

**If p > 0.05: first series cannot be used to predict second series**

‘params_ftest’, ‘ssr_ftest’ are based on F distribution

‘ssr_chi2test’, ‘lrtest’ are based on chi-square distribution

Dataframes indicate whether the arrondissement in the row Granger causes the arrondissement in the column

*Note that the granger test does not differ whether applied on absolute or on relative cases:*

In [ ]:
granger_test(ts_arr_cases, 'Antwerpen', 'Turnhout', lag=1)

In [ ]:
granger_test(ts_arr_rel_cases, 'Antwerpen', 'Turnhout', lag=1)

In [14]:
## compare all arrondissements
stat_granger, p_granger = test_df(granger_test, ts_arr_rel_cases, ts_arr_cases.columns)
p_granger[p_granger<0.05]

,Aalst,Aarlen,Aat,Antwerpen,Bastenaken,Bergen,Borgworm,Brugge,Brussel-Hoofdstad,Charleroi,...,Roeselare,Sint-Niklaas,Thuin,Tielt,Tongeren,Turnhout,Verviers,Veurne,Virton,Zinnik
Aalst,NaN,NaN,0.0292723,NaN,0.023387,0.00568999,0.000547222,NaN,NaN,0.0391867,...,0.0423309,NaN,NaN,NaN,NaN,NaN,0.00618062,6.05584e-06,0.0196356,0.00645552
Aarlen,NaN,NaN,NaN,0.0258789,NaN,NaN,NaN,NaN,0.0178111,NaN,...,NaN,NaN,NaN,NaN,0.045272,NaN,NaN,NaN,NaN,NaN
Aat,NaN,NaN,NaN,NaN,NaN,0.0324263,NaN,NaN,NaN,0.0166994,...,0.0183793,NaN,0.00154371,NaN,NaN,NaN,NaN,0.0402155,NaN,0.00669524
Antwerpen,0.000829843,0.0234465,NaN,NaN,0.0417663,0.0157091,0.00502602,0.0024752,0.00432131,0.0194807,...,7.05674e-08,0.022159,0.00403863,8.70535e-05,NaN,0.016403,0.000146036,0.00944017,NaN,8.42123e-05
Bastenaken,0.00705705,5.84239e-05,0.0045592,0.00411236,NaN,0.045438,0.00444514,0.00140506,0.0045922,0.000162622,...,0.00112392,0.000248149,9.5943e-05,0.00282771,0.000653185,0.0322026,NaN,0.0141439,0.000385295,0.00266307
Bergen,NaN,NaN,0.013531,NaN,NaN,NaN,0.0272434,NaN,NaN,NaN,...,0.0110753,0.019909,NaN,0.012849,NaN,NaN,NaN,0.00228771,0.0076914,NaN
Borgworm,NaN,NaN,NaN,NaN,NaN,0.000754191,NaN,NaN,NaN,0.00437788,...,0.0485983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000236765
Brugge,0.0333398,NaN,NaN,NaN,NaN,NaN,0.014524,NaN,0.00959845,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0139111,NaN,NaN,NaN
Brussel-Hoofdstad,0.000823887,0.00479219,1.31815e-05,3.98766e-05,0.000738513,0.0017726,1.48352e-06,0.0191583,NaN,0.000943282,...,0.00817842,0.000469057,0.000810188,0.00113659,0.0176049,0.0102265,0.000133762,0.00024519,0.000408806,0.000407165
Charleroi,0.0206159,NaN,NaN,NaN,NaN,0.00141482,0.000466463,NaN,0.00722944,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.30856e-06,NaN,NaN,0.00476098


## Dynamic time warping

https://pypi.org/project/dtw-python/